In [1]:
###-Repositorio estadistica informatica pregrado UNALM-### 

#Trabajo Final → AVANCE.
#1. Relación de repositorios con sus respectivos url
#2. Automatización de la descarga de la información de un repositorio.
#3. Estructuración de la información descargada en formato csv

#repositorios

#1. UNALM Pregrado #http://repositorio.lamolina.edu.pe/handle/20.500.12996/26/recent-submissions
#2. PUCP Posgrado #https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/757/recent-submissions
#3. UNALM Posgrado #http://repositorio.lamolina.edu.pe/handle/20.500.12996/329


## Repositorio 1 PUCP posgrado 
url = 'https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/757/recent-submissions'

#paquetes 
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

import ssl
context = ssl._create_unverified_context() #la pagina pide un socket seguro 

#Las tesis se encuentran en 4 paginas distinas 

#devuelve la url de la siguiente pagina 
def next_page(url):
    page = urllib.request.urlopen(url,context = context )
    soup = BeautifulSoup(page, "html.parser")
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('\?offset.*','',url)
    n_url = url + url_diff
    return n_url

URL = []
url1 = url

#devuelve las url de todas las paginas 
def all_pages(url,URL):
    
    page = urllib.request.urlopen(url,context = context)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    if last == None: 
        url = next_page(url)
        all_pages(url,URL)
    else: 
        return URL 
    return URL 
all_pages(url1,URL)

###Almacenar links de las tesis 

handles = [] 

#las url de las tesis se diferencian por el codigo handle/20.500.12404/***** <-- diferencia 
for urls in URL: 
    page = urllib.request.urlopen(urls,context = context)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12404/).+?(?=\")',lines) #handle unico a cada tesis (#12996)
        handles.append(link)
       
## Links completos 

#creando las url especificas de cada tesis 
links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12404/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

#agregando ?show=full a las url se muestra informacion mas detallada de cada tesis 
links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')



In [5]:
## almacenando en diccionario 

rep_PUCP = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'resumenes': [],
        'años' : [] }
 
def get_all(links_full,reps_PUCP):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i],context = context)
        soup = BeautifulSoup(page, "html.parser")
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+'\n'+full_t
            
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        resumen = re.findall(r'(?<=<tr class=\"ds-table-row even \">\n<td class=\"label-cell\">dc\.description\.abstract</td><td class=\"word-break\">)[\s\S]*?(?=es_ES)',full_t)
        año = re.findall(r'(?<=>dc\.date\.issued</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        
        rep_PUCP['unis'].append(uni[0])
        rep_PUCP['titulos'].append(titulo[0])     
        rep_PUCP['nombres'].append(nombre[0])
        rep_PUCP['grados'].append(grado[0]) 
        
        if asesor == []: 
            rep_PUCP['asesores'].append('Sin asesor')
        else:
            rep_PUCP['asesores'].append(asesor[0])
        if resumen == []: 
            rep_PUCP['resumenes'].append('Sin resumen')
        else:
            rep_PUCP['resumenes'].append(resumen[0])
        rep_PUCP['años'].append(año[0])
    return reps_PUCP

universities = get_all(links_full,rep_PUCP)


In [6]:
df_PUCP = pd.DataFrame(universities)
df_PUCP.to_csv('tesisPUCP.csv', index=False,encoding='utf-8-sig',
          header = ['Institución','Título de la tesis', 'Nombre del tesista', 'Grado', 'Nombre del asesor', 'Resumen','Año'])

df_PUCP_read = pd.read_csv('tesisPUCP.csv')
df_PUCP_read

,Institución,Título de la tesis,Nombre del tesista,Grado,Nombre del asesor,Resumen,Año
0,Pontificia Universidad Católica del Perú. Escu...,Estimación del ciclo financiero utilizando mét...,"Atoche Murrieta, Lilian Katherine",Magíster en Estadística,"Camiz, Sergio",The objective of this thesis is to identify th...,2022-09-22
1,Pontificia Universidad Católica del Perú. Escu...,Método para la fusión de categorías usando téc...,"Farro Diaz, Victor Daniel",Magíster en Estadística,"Bayes Rodriguez, Cristian Luis","En la actualidad, muchas organizaciones dispon...",2022-04-28
2,Pontificia Universidad Católica del Perú. Escu...,Redes neuronales convolucionales para datos co...,"Cotacallapa Amanqui, Pavel Arturo",Magíster en Estadística,"Benites Sanchez, Luis Enrique",En muchas situaciones prácticas es necesario e...,2022-04-07
3,Pontificia Universidad Católica del Perú. Escu...,Modelo de regresión cuantílica para respuestas...,"Manrique Urbina, Justo Andrés",Magíster en Estadística,"Bayes Rodriguez, Cristian Luis",La presente tesis propone un modelo de regresi...,2022-03-21
4,Pontificia Universidad Católica del Perú. Escu...,Clasiﬁcación de riesgo para frecuencias y seve...,"Hernández Bello, Diana Patricia",Magíster en Estadística,"Valdivieso Serrano, Luis Hilmar","En la tariﬁcación de seguros generales, en par...",2022-03-16
...,...,...,...,...,...,...,...
56,Pontificia Universidad Católica del Perú. Escu...,Modelos de regresión binaria Skew probit para ...,"Pantoja Marin, Luis",Magíster en Estadística,"Bazán Guzmán, Jorge Luis",La presente investigación se fundamenta en el ...,2013-02-05
57,Pontificia Universidad Católica del Perú. Escu...,Modelo de Rasch dicotómico con aplicación a la...,"Chincaro Del Coral, Omar Antonio",Magíster en Estadística,"Bazán Guzmán, Jorge Luis",En investigaciones de origen cuantitativo gene...,2012-08-17
58,Pontificia Universidad Católica del Perú. Escu...,Estimación no paramétrica en un proceso de Mar...,"Requena Espinoza, Genaro",Magíster en Estadística,"Doig Camino, Elizabeth","En el presente trabajo, se estudian las propie...",2012-08-16
59,Pontificia Universidad Católica del Perú. Escu...,Modelos testlet logísticos y logísticos de exp...,"Flores Ari, Sandra Elizabeth",Magíster en Estadística,"Bazán Guzmán, Jorge Luis",Los modelos de Teoría de Respuesta al Item (TR...,2012-08-16
